<img align="right" src="https://raw.githubusercontent.com/dcblum/OpenStreetMap/master/IrvineMapBoundaries.PNG" title="Irvine Map Extract Area">

# OpenStreetMap Case Study with SQL

### Map Area: Irvine, California

City Boundaries:
* https://www.openstreetmap.org/relation/114485

Map Extract Website:
* https://mapzen.com/data/metro-extracts-alt/

The Map Extract data extends beyond Irvine City limits in the shape of a rectangle. For the purpose of this dataset the any data in the given rectangular picture is considered to be in Irvine.

#### Contents
 - [Problems Encountered](#problems-encountered)

### Problems Encountered:

After running the data agaisnt the given data.py file, a few problems were found:
* Inconsistent use of abbreviations: ex. "Rd.", "Rd", and "Road" to all indicate "Road"
* Use of "St." indicating both "Street" and "Saint"
* Unicode street and postcode data containing (street = 'Rue Jean Jaur\xe8s' and postcode = '92626\xad)
* Postcode data contaning non-integer characters such as letters and hyphens: ex. "CA 92701" and "92610-3437"

 Number of Unique Users:
        2315

### Street Names:

The OSM file of Irvine was audited via file audit.py to help correct inconsistent use of abbreviations. To help filter results, "expected" 

In [ ]:
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", "Trail", "Parkway", "Ridge", 
            "Commons", "Way", "Vista", "Terrace", "Circle", "Glen", "Grove", "Hill", "South", "East", "West", "North", "Tree", 
            "Valley", "Creek", "Canyon", "Brook", "Bloom", "Springs", "Verde", "Arbor", "Center", "Loop"]